In [1]:
import warnings
import pandas as pd
import numpy as np
import lightgbm
import re
import unicodedata

from functools import partial
from tqdm.auto import tqdm

from sklearn.model_selection import cross_validate, KFold
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.compose import (
    make_column_transformer,
    TransformedTargetRegressor,
)
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OrdinalEncoder as SKOrdinalEncoder
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.base import TransformerMixin, BaseEstimator

from category_encoders.target_encoder import TargetEncoder

warnings.simplefilter("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# little snippet to normalize text by removing accents
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)


In [3]:
LAT_LONG_COLUMNS = ["approximate_latitude", "approximate_longitude"]

N_CV_FOLDS = 10
SCORING = "neg_mean_absolute_percentage_error"
USE_IMAGE_EMBEDDING = True
USE_DVF = True
USE_SALARIES = True
USE_MAGIC = True

# Ideas on simple features

In [4]:
df_train = pd.read_csv("../data/X_train.csv")
df_test = pd.read_csv("../data/X_test.csv")
y_train = pd.read_csv("../data/y_train.csv")

df_train["fold"] = "train"
df_test["fold"] = "test"

df_all = pd.concat([df_train, df_test], axis=0)
data_all = pd.merge(df_all, y_train, on="id_annonce", how="left")

In [5]:
data_all.head()

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,energy_performance_category,ghg_value,ghg_category,exposition,nb_rooms,nb_bedrooms,nb_bathrooms,nb_parking_places,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,fold,price
0,35996577,appartement,43.643880,7.117183,villeneuve-loubet,6270,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,train,355000.0
1,35811033,appartement,45.695757,4.895610,venissieux,69200,90.0,3.0,NaN,223.0,D,52.0,E,NaN,5.0,4.0,NaN,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,190000.0
2,35731841,maison,47.966791,-1.220451,moutiers,35130,61.0,NaN,370.0,NaN,NaN,NaN,NaN,Sud,2.0,1.0,NaN,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,39000.0
3,35886765,maison,47.289292,-1.878805,cordemais,44360,142.0,NaN,764.0,217.0,D,44.0,E,NaN,4.0,3.0,NaN,0.0,1.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,train,299000.0
4,35781137,appartement,45.718992,4.844234,lyon-7eme,69007,88.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,1.0,0.0,1.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,train,478000.0


# Simple transforms

In [6]:
data_all["departement"] = data_all["postal_code"].astype(
    str).str[:-3].str.zfill(2)

data_all["post_code_suffix"] = data_all["postal_code"] % 1000

data_all["city_enc"] = data_all["city"]

# a few basic ratios
data_all["nb_bedrooms_over_rooms"] = data_all["nb_bedrooms"].div(
    data_all["nb_rooms"].replace(0.0, np.nan)
)
data_all["size_over_rooms"] = data_all["size"].div(
    data_all["nb_rooms"].replace(0.0, np.nan)
)
data_all["size_over_landsize"] = data_all["size"].div(
    data_all["land_size"].replace(0.0, np.nan)
)
data_all["sqrt_size"] = np.sqrt(data_all["size"])

# interaction columns
data_all["type_rooms"] = data_all["property_type"] + data_all["nb_rooms"].clip(
    None, 8.0
).fillna("nan").astype(str)
data_all["type_bedrooms"] = data_all["property_type"] + data_all["nb_bedrooms"].clip(
    None, 6.0
).fillna("nan").astype(str)
data_all["city_property_type"] = (
    data_all["city"] + data_all["property_type"]).astype(str)
data_all["departement_property_type"] = (
    data_all["departement"] + data_all["property_type"]).astype(str)
data_all["city_nb_rooms"] = (
    data_all["city"] + data_all["nb_rooms"].astype(str)).astype(str)
data_all["departement_nb_rooms"] = (
    data_all["departement"] + data_all["nb_rooms"].astype(str)).astype(str)

data_all = data_all.rename(columns={"nb_terraces": "has_a_terrace"})

data_all["diff_size_per_departement"] = (
    data_all["size"] - 
    data_all.groupby(["departement", "property_type"])["size"].transform("mean")
)
data_all["diff_size_per_city"] = (
    data_all["size"] - 
    data_all.groupby(["city", "property_type"])["size"].transform("mean")
)
data_all["diff_landsize_per_city"] = (
    data_all["land_size"] - 
    data_all.groupby(["city", "property_type"])["land_size"].transform("mean"))

# drop useless indicators
data_all = data_all.drop(["upper_floors", "last_floor"], axis=1)

## Incorporate image prediction as a feature


In [7]:
if USE_IMAGE_EMBEDDING:
    image_pred_mix_train = pd.read_csv("../images_features_effnet_resnet_train.csv")
    image_pred_mix_test = pd.read_csv("../images_features_effnet_resnet_test.csv")
    
    image_pred_mix = pd.concat((image_pred_mix_train, image_pred_mix_test), axis=0)

    data_all = pd.merge(data_all, image_pred_mix[["id_annonce", "price_pred"]])

In [8]:
if USE_DVF:
    dvf = pd.read_csv("../data/dvf_codepostal.csv")

    data_all = pd.merge(
        data_all,
        dvf.rename(columns={"Code postal": "postal_code"}),
        on="postal_code",
        how="left"
    )

    dvf = pd.read_csv("../data/dvf_codepostal_pieces.csv")

    data_all = pd.merge(
        data_all,
        dvf.rename(columns={
            "Code postal": "postal_code",
            "Nombre pieces principales": "nb_rooms"
        }),
        on=["postal_code", "nb_rooms"],
        how="left",
        suffixes=("_pc", "_rooms")
    )

In [9]:
if USE_SALARIES:
    # add salary per city
    df_salaires_villes = pd.read_csv("../data/salaires_villes.csv")
    df_salaires_villes["city"] = df_salaires_villes["city"].apply(
        strip_accents)

    data_all = pd.merge(
        data_all,
        df_salaires_villes[["city", "SNHM19"]],
        on="city",
        how='left')

    # add salary per departements
    df_salaires_departements = pd.read_csv("../data/salaires_departements.csv")
    data_all = pd.merge(
        data_all,
        df_salaires_departements[["departement", "SNHM19_dep"]],
        on="departement",
        how='left'
    )

# Modeling

In [10]:
CATEGORICAL_COLUMNS = [
    "property_type",
    "exposition",
    "has_a_balcony",
    "has_a_terrace",
    "has_a_cellar",
    "has_a_garage",
    "has_air_conditioning",
    "energy_performance_category",
    "ghg_category",
    "type_rooms",
    "type_bedrooms",
    "city",
]

for col in CATEGORICAL_COLUMNS:
    data_all[col] = data_all[col].astype("category")

In [11]:
data_train = data_all.query("fold=='train'").drop("fold", axis=1)
data_test = data_all.query("fold=='test'").drop("fold", axis=1)

In [12]:
# quick patch to add normalization capability

from category_encoders.ordinal import OrdinalEncoder
import category_encoders.utils as util


class TargetEncoder(BaseEstimator, util.TransformerWithTargetMixin):
    def __init__(
        self,
        verbose=0,
        cols=None,
        drop_invariant=False,
        return_df=True,
        handle_missing="value",
        handle_unknown="value",
        min_samples_leaf=1,
        smoothing=1.0,
        normalize=False,
    ):
        self.return_df = return_df
        self.drop_invariant = drop_invariant
        self.drop_cols = []
        self.verbose = verbose
        self.cols = cols
        self.ordinal_encoder = None
        self.min_samples_leaf = min_samples_leaf
        self.smoothing = float(
            smoothing
        )  # Make smoothing a float so that python 2 does not treat as integer division
        self._dim = None
        self.mapping = None
        self.handle_unknown = handle_unknown
        self.handle_missing = handle_missing
        self._mean = None
        self.feature_names = None
        self.normalize = normalize

    def fit(self, X, y, **kwargs):
        """Fit encoder according to X and y.

        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.

        Returns
        -------
        self : encoder
            Returns self.

        """

        # unite the input into pandas types
        X = util.convert_input(X)
        y = util.convert_input_vector(y, X.index)

        if X.shape[0] != y.shape[0]:
            raise ValueError(
                "The length of X is "
                + str(X.shape[0])
                + " but length of y is "
                + str(y.shape[0])
                + "."
            )

        self._dim = X.shape[1]

        # if columns aren't passed, just use every string column
        if self.cols is None:
            self.cols = util.get_obj_cols(X)
        else:
            self.cols = util.convert_cols_to_list(self.cols)

        if self.handle_missing == "error":
            if X[self.cols].isnull().any().any():
                raise ValueError("Columns to be encoded can not contain null")

        self.ordinal_encoder = OrdinalEncoder(
            verbose=self.verbose,
            cols=self.cols,
            handle_unknown="value",
            handle_missing="value",
        )
        self.ordinal_encoder = self.ordinal_encoder.fit(X)
        X_ordinal = self.ordinal_encoder.transform(X)
        self.mapping = self.fit_target_encoding(X_ordinal, y)

        X_temp = self.transform(X, override_return_df=True)
        self.feature_names = list(X_temp.columns)

        if self.drop_invariant:
            self.drop_cols = []
            X_temp = self.transform(X)
            generated_cols = util.get_generated_cols(X, X_temp, self.cols)
            self.drop_cols = [
                x for x in generated_cols if X_temp[x].var() <= 10e-5]
            try:
                [self.feature_names.remove(x) for x in self.drop_cols]
            except KeyError as e:
                if self.verbose > 0:
                    print(
                        "Could not remove column from feature names."
                        "Not found in generated cols.\n{}".format(e)
                    )

        return self

    def fit_target_encoding(self, X, y):
        mapping = {}

        for switch in self.ordinal_encoder.category_mapping:
            col = switch.get("col")
            values = switch.get("mapping")

            prior = self._mean = y.mean()

            if self.normalize:
                normalizer = X["size"].replace(0.0, np.nan).values
            else:
                normalizer = 1
            stats = (y / normalizer).replace([-np.inf, np.inf],
                                             np.nan).groupby(X[col]).agg(["count", "mean"])

            smoove = 1 / (
                1 + np.exp(-(stats["count"] -
                             self.min_samples_leaf) / self.smoothing)
            )
            smoothing = prior * (1 - smoove) + stats["mean"] * smoove
            smoothing[stats["count"] == 1] = prior

            if self.handle_unknown == "return_nan":
                smoothing.loc[-1] = np.nan
            elif self.handle_unknown == "value":
                smoothing.loc[-1] = prior

            if self.handle_missing == "return_nan":
                smoothing.loc[values.loc[np.nan]] = np.nan
            elif self.handle_missing == "value":
                smoothing.loc[-2] = prior

            mapping[col] = smoothing

        return mapping

    def transform(self, X, y=None, override_return_df=False):
        """Perform the transformation to new categorical data.

        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
        y : array-like, shape = [n_samples] when transform by leave one out
            None, when transform without target info (such as transform test set)

        Returns
        -------
        p : array, shape = [n_samples, n_numeric + N]
            Transformed values with encoding applied.

        """

        if self.handle_missing == "error":
            if X[self.cols].isnull().any().any():
                raise ValueError("Columns to be encoded can not contain null")

        if self._dim is None:
            raise ValueError(
                "Must train encoder before it can be used to transform data."
            )

        # unite the input into pandas types
        X = util.convert_input(X)

        # then make sure that it is the right size
        if X.shape[1] != self._dim:
            raise ValueError(
                "Unexpected input dimension %d, expected %d"
                % (
                    X.shape[1],
                    self._dim,
                )
            )

        # if we are encoding the training data, we have to check the target
        if y is not None:
            y = util.convert_input_vector(y, X.index)
            if X.shape[0] != y.shape[0]:
                raise ValueError(
                    "The length of X is "
                    + str(X.shape[0])
                    + " but length of y is "
                    + str(y.shape[0])
                    + "."
                )

        if not list(self.cols):
            return X

        X = self.ordinal_encoder.transform(X)

        if self.handle_unknown == "error":
            if X[self.cols].isin([-1]).any().any():
                raise ValueError("Unexpected categories found in dataframe")

        X = self.target_encode(X)

        if self.drop_invariant:
            for col in self.drop_cols:
                X.drop(col, 1, inplace=True)

        if self.return_df or override_return_df:
            return X
        else:
            return X.values

    def target_encode(self, X_in):
        X = X_in.copy(deep=True)

        for col in self.cols:
            X[col] = X[col].map(self.mapping[col])

        return X

    def get_feature_names(self):
        """
        Returns the names of all transformed / added columns.

        Returns
        -------
        feature_names: list
            A list with all feature names transformed or added.
            Note: potentially dropped features are not included!

        """

        if not isinstance(self.feature_names, list):
            raise ValueError(
                "Must fit data first. Affected feature names are not known before."
            )
        else:
            return self.feature_names
        
    def get_feature_names_out(self):
        """
        Returns the names of all transformed / added columns.

        Returns
        -------
        feature_names: list
            A list with all feature names transformed or added.
            Note: potentially dropped features are not included!

        """

        if not isinstance(self.feature_names, list):
            raise ValueError(
                "Must fit data first. Affected feature names are not known before."
            )
        else:
            return self.feature_names

In [13]:
ordinal_encoder = make_column_transformer(
    (
        SKOrdinalEncoder(
            handle_unknown="use_encoded_value",
            unknown_value=np.nan
        ),
        CATEGORICAL_COLUMNS,
    ),
    (
        TargetEncoder(verbose=1, normalize=True),
        [
             "city_enc", 
             "departement", 
             "city_property_type",
             "departement_property_type", 
             "city_nb_rooms", 
             "departement_nb_rooms", 
             "size"
        ],
    ),
    remainder="passthrough",
)

In [14]:
# magic feature : compute mean price of n-closest houses
class NNTransformer(TransformerMixin):
    """Compute the mean price per square-meter in a given radius."""
    def __init__(self, n_neighbors, columns, normalize=False):
        self.n_neighbors = n_neighbors
        self.columns = columns
        self.normalize = normalize

    def fit(self, X_train, y):
        self.nearest_neighbors = NearestNeighbors(n_neighbors=self.n_neighbors)

        data_train = X_train[self.columns].fillna(0)
        self.nearest_neighbors.fit(data_train)

        if self.normalize:
            # normalize prices by size to have the price per square meters
            self.prices = (
                (y / X_train["size"])
                .replace([-np.inf, np.inf], np.nan)
                .values
            )
        else:
            self.prices = y.values

        return self

    def transform(self, X_all):
        data_all = X_all[self.columns].fillna(0)
        neighbors_id = self.nearest_neighbors.kneighbors(data_all)[1]
        
        neighbors_price = np.array(
            [np.nanmean(self.prices[neighbors_id[:, 1:]], axis=1)]
        ).T
        
        return neighbors_price
    
    def get_feature_names_out(self, input_features):
        return f"nearest_{self.n_neighbors}_mean_price"

In [15]:
class RadiusTransformer(TransformerMixin, BaseEstimator):
    """Count the number of houses in the dataset in a given radius."""
    def __init__(self, radius, columns):
        self.radius = radius
        self.columns = columns

    def fit(self, X_train, y):
        self.rn = NearestNeighbors()
        
        data_train = X_train[self.columns]
        self.rn.fit(data_train)
        
        return self

    def transform(self, X_all):
        data_all = X_all[self.columns]
        
        nn_kneighbors = self.rn.radius_neighbors(data_all, radius=self.radius)[1]
        
        return np.array([[len(neigh) for neigh in nn_kneighbors]]).T
    
    def get_feature_names_out(self, input_features):
        return f"radius_{self.radius}_density"

In [16]:
if USE_MAGIC:
    ranges_nnt = range(3, 11, 1)
    ranges_rt = range(0, 8, 1)
else:
    ranges_nnt = []
    ranges_rt = []
    
feats = make_union(
    ordinal_encoder,
    *(
        NNTransformer(
            n_neighbors=2 ** i,
            columns=LAT_LONG_COLUMNS,
            normalize=True
        )
        for i in ranges_nnt
    ),
    *(RadiusTransformer(1 / (2 ** i), LAT_LONG_COLUMNS) for i in ranges_rt),
)

In [17]:
data_train.head(2)

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,energy_performance_category,ghg_value,ghg_category,exposition,nb_rooms,nb_bedrooms,nb_bathrooms,nb_parking_places,nb_boxes,nb_photos,has_a_balcony,has_a_terrace,has_a_cellar,has_a_garage,has_air_conditioning,price,departement,post_code_suffix,city_enc,nb_bedrooms_over_rooms,size_over_rooms,size_over_landsize,sqrt_size,type_rooms,type_bedrooms,city_property_type,departement_property_type,city_nb_rooms,departement_nb_rooms,diff_size_per_departement,diff_size_per_city,diff_landsize_per_city,price_pred,Surface reelle bati_pc,Surface terrain_pc,Valeur fonciere_pc,Nombre pieces principales,prix_metre_carre_pc,prix_metre_carre_terrain_pc,Surface reelle bati_rooms,Surface terrain_rooms,Valeur fonciere_rooms,prix_metre_carre_rooms,prix_metre_carre_terrain_rooms,SNHM19,SNHM19_dep
0,35996577,appartement,43.643880,7.117183,villeneuve-loubet,6270,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,355000.0,06,270,villeneuve-loubet,0.666667,21.0,NaN,7.937254,appartement3.0,appartement2.0,villeneuve-loubetappartement,06appartement,villeneuve-loubet3.0,063.0,-1269.792230,-1622.600000,NaN,243192.935182,74.418048,601.351351,332958.364364,1.318854,5422.001419,7721.291671,68.088000,501.333333,350521.26424,5164.200709,1978.664430,16.478742,15.48594
1,35811033,appartement,45.695757,4.895610,venissieux,69200,90.0,3.0,NaN,223.0,D,52.0,E,NaN,5.0,4.0,NaN,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,190000.0,69,200,venissieux,0.800000,18.0,NaN,9.486833,appartement5.0,appartement4.0,venissieuxappartement,69appartement,venissieux5.0,695.0,-1361.143564,-228.906977,NaN,170271.593424,90.151149,771.232456,305309.423628,1.777011,6152.318233,2445.067978,98.508475,579.608696,252607.00000,2589.887880,851.321408,12.531494,16.72526


In [18]:
X = data_train.drop("price", axis=1)
y = data_train["price"]

n_categorical = (data_train.dtypes.iloc[:-1] == "category").values.sum()
categorical_mask = list(range(n_categorical))

In [19]:
%%time

hgb = lightgbm.LGBMRegressor(
    random_state=42,
    categorical_features=categorical_mask,
    n_estimators=2_000,
    learning_rate=0.025,
    num_leaves=256,
    min_child_samples=10,
    colsample_bytree=0.25,
)

hist_native = make_pipeline(
    feats,
    TransformedTargetRegressor(
        regressor=hgb,
        func=np.log,
        inverse_func=np.exp,
    ),
    verbose=1
)

results = cross_validate(
    hist_native,
    X,
    y,
    cv=N_CV_FOLDS,
    scoring=SCORING,
    return_estimator=True,
    return_train_score=True,
    verbose=1,
    n_jobs=-1,
)

print(f"Mean over val set = {-results['test_score'].mean():.1%}")
print(f"Mean over train set = {-results['train_score'].mean():.1%}\n")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  4.1min remaining:  2.7min


Mean over val set = 22.4%
Mean over train set = 2.3%

CPU times: user 9.17 s, sys: 2.73 s, total: 11.9 s
Wall time: 5min 49s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  5.8min finished


# Prediction

In [20]:
data_test = data_test[~data_test.duplicated(subset="id_annonce")]

predictions = list()

for estimator in tqdm(results["estimator"]):
    raw_pred = estimator.predict(data_test.drop("price", axis=1))
    pred = pd.Series(raw_pred, index=data_test["id_annonce"])
    predictions.append(pred)

predictions = pd.concat(predictions, axis=1)

  0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
SAVE = False

if SAVE:
    k_rd = np.random.randint(0, 1_000) # kind of a hash of the version
    predictions.to_csv(f"../predictions/{k_rd}.csv", index=False)

    print(f"Saved to path: ../predictions/{k_rd}.csv")